In [1]:
from __future__ import print_function, division

In [22]:
import matplotlib.pyplot as plt
%matplotlib inline 
import numpy as np

import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dropout, Dense, Activation, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import AveragePooling2D
from keras import optimizers
from keras import initializers
from keras import regularizers

In [3]:
from keras.datasets import cifar10

# 10 classes for cifar10.
nb_classes = 10

# Download and load the data.
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Convert to float datatype.
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

# Normalize so the max value is 1.
X_train /= 255
X_test /= 255

# Change to 1-hot encoding.
Y_train = keras.utils.np_utils.to_categorical(y_train, nb_classes)
Y_test = keras.utils.np_utils.to_categorical(y_test, nb_classes)

In [4]:
# Check our shapes.
print(X_train.shape)
print(X_test.shape)

print(Y_train.shape)
print(Y_test.shape)

(50000, 32, 32, 3)
(10000, 32, 32, 3)
(50000, 10)
(10000, 10)


In [5]:
# Subtract by the mean pixel, so image data is centered around 0.
# This differs from the paper in which, 
# "images were whitened and contrast normalized following Goodfellow et al. (2013)."
mean_px = X_train.mean(axis=0).mean(axis=0).mean(axis=0)
X_train -= mean_px

In [16]:
?Conv2D

In [18]:
def conv(x, nb_filters, kernel_size=(3,3), strides=(1,1), padding='same', 
         kernel_initializer=initializers.he_uniform(),  # Not sure how weights were initialized.
         kernel_regularizer=regularizers.l2(0.001),  # "all models were regularized with weight decay \lambda=0.001"
         activation='relu',  # Table 1,2    
        ):
    """Convolutional layer"""

    x = Conv2D(nb_filters, kernel_size=kernel_size, strides=strides, 
           padding=padding, kernel_initializer=kernel_init, 
           kernel_regularizer=reg, activation=activation)(x)

    return x

In [30]:
# Model C from Table 1 and 2.
inputs = Input(shape=(32, 32, 3))
x = Dropout(0.2)(inputs) # "dropout probabilities were 20% for dropping out inputs ..."

x = conv(x, 96)
x = conv(x, 96)
x = conv(x, 96, strides=(2,2))
x = Dropout(0.5)(x)  # "dropout ... after the layer replacing the pooling layer ... and 50% ..."

x = conv(x, 192)
x = conv(x, 192)
x = conv(x, 192, strides=(2,2))
x = Dropout(0.5)(x)

x = conv(x, 192)
x = conv(x, 192, kernel_size=(1,1), padding='valid')
x = conv(x, nb_filters=nb_classes, kernel_size=(1,1), padding='valid')

x = GlobalAveragePooling2D()(x)
preds = Activation('softmax')(x)

model = Model(inputs=inputs, outputs=preds)
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [31]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 32, 32, 96)        2688      
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 32, 32, 96)        83040     
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 16, 16, 96)        83040     
_________________________________________________________________
dropout_17 (Dropout)         (None, 16, 16, 96)        0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 16, 16, 192)       166080    
__________

In [32]:
model.fit(X_train, Y_train, epochs=1, validation_split=0.2, shuffle=True, verbose=1)

Train on 40000 samples, validate on 10000 samples
Epoch 1/1
40000/40000 [==============================] - 138s - loss: 3.8780 - acc: 0.2823 - val_loss: 3.3183 - val_acc: 0.3398